In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string

In [ ]:
# load the dataset
df = pd.read_csv('/content/drive/MyDrive/ProjectFakeNews/WELFake_Dataset.csv')

In [ ]:
df.head() # first 5 entries in the dataset

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [ ]:
df.columns = ["id","title","text","label"] # name the columns

In [ ]:
df.head()

,id,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
df.drop(labels=['id','title'], axis='columns', inplace=True) # drop unnecessary columns

In [ ]:
def preprocess(s): # convert all the data in text to string
    s = str(s)
    return s

In [ ]:
df['text'] = df['text'].apply(preprocess)

In [ ]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_md",disable=["ner", "parser"])

In [ ]:
text = df.text.to_list()

In [ ]:
# using the spacy library to lemmatize after tokenizing, and removing stopwords, space, currency and special characters
ls = []
for d in nlp.pipe(text,n_process=-1):
    docu = []
    for token in d.doc:
        if token.is_bracket or token.is_space or token.is_currency or token.is_stop or token.is_punct:
            continue
        else:
            docu.append(token.lemma_)
    ls.append(docu)  

In [ ]:
lemm = pd.Series(ls) # convert the list to series

In [ ]:
df['lemm_sent'] = lemm # add the lemmatized tokens back to the dataframe

In [ ]:
df.head()

,text,label,lemm_sent
0,No comment is expected from Barack Obama Membe...,1,"[comment, expect, Barack, Obama, member, FYF91..."
1,Did they post their votes for Hillary already?,1,"[post, vote, Hillary]"
2,"Now, most of the demonstrators gathered last ...",1,"[demonstrator, gather, night, exercise, consti..."
3,A dozen politically active pastors came here f...,0,"[dozen, politically, active, pastor, come, pri..."
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,"[rs-28, sarmat, missile, dub, Satan, 2, replac..."


In [ ]:
# function to convert the token to lower case
def low(s):
    ls = []
    for word in s:
        ls.append(word.lower())
    return ls

In [ ]:
df.lemm_sent = df.lemm_sent.apply(low) # applying the function low

In [ ]:
df.head()

,text,label,lemm_sent
0,No comment is expected from Barack Obama Membe...,1,"[comment, expect, barack, obama, member, fyf91..."
1,Did they post their votes for Hillary already?,1,"[post, vote, hillary]"
2,"Now, most of the demonstrators gathered last ...",1,"[demonstrator, gather, night, exercise, consti..."
3,A dozen politically active pastors came here f...,0,"[dozen, politically, active, pastor, come, pri..."
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,"[rs-28, sarmat, missile, dub, satan, 2, replac..."


In [ ]:
from gensim.models.phrases import Phraser, ENGLISH_CONNECTOR_WORDS, Phrases
import gensim.downloader as api

In [ ]:
# preparing data for phraser
data = []
for ls in df["lemm_sent"]:
    data.append(ls)

###### Training the Bigram and trigram phraser model in the gensim library with our dataset

In [ ]:
bigram = Phrases(data,min_count=1,threshold=2,connector_words=ENGLISH_CONNECTOR_WORDS)

In [ ]:
trigram = Phrases(bigram[data],threshold=3)

In [ ]:
bigram = Phraser(bigram)
trigram = Phraser(trigram)

The phraser connectes words that are most likely one word, for example ['new','york'] --> ['new_york'].
Bigram connects 2 words and trigram connects 3

In [ ]:
# defining a function to apply the pharaser on our tokenized data
def rephrase(s):
    s = bigram[s]
    s = trigram[s]
    return s

In [ ]:
df['rephrased'] = df['lemm_sent'].apply(rephrase)

In [ ]:
df.head()

,text,label,lemm_sent,rephrased
0,No comment is expected from Barack Obama Membe...,1,"[comment, expect, barack, obama, member, fyf91...","[comment, expect, barack_obama, member, fyf911..."
1,Did they post their votes for Hillary already?,1,"[post, vote, hillary]","[post, vote_hillary]"
2,"Now, most of the demonstrators gathered last ...",1,"[demonstrator, gather, night, exercise, consti...","[demonstrator_gather, night, exercise_constitu..."
3,A dozen politically active pastors came here f...,0,"[dozen, politically, active, pastor, come, pri...","[dozen, politically_active, pastor, come, priv..."
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,"[rs-28, sarmat, missile, dub, satan, 2, replac...","[rs-28_sarmat, missile_dub, satan_2, replace_s..."


In [ ]:
# function to join the tokens to form sentences again. 
def joinner(ls):
    s = ' '
    s = s.join(ls)
    return s

In [ ]:
df['joined_lemm'] = df['rephrased'].apply(joinner)

In [ ]:
df.head()

,text,label,lemm_sent,rephrased,joined_lemm
0,No comment is expected from Barack Obama Membe...,1,"[comment, expect, barack, obama, member, fyf91...","[comment, expect, barack_obama, member, fyf911...",comment expect barack_obama member fyf911 fuky...
1,Did they post their votes for Hillary already?,1,"[post, vote, hillary]","[post, vote_hillary]",post vote_hillary
2,"Now, most of the demonstrators gathered last ...",1,"[demonstrator, gather, night, exercise, consti...","[demonstrator_gather, night, exercise_constitu...",demonstrator_gather night exercise_constitutio...
3,A dozen politically active pastors came here f...,0,"[dozen, politically, active, pastor, come, pri...","[dozen, politically_active, pastor, come, priv...",dozen politically_active pastor come private_d...
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,"[rs-28, sarmat, missile, dub, satan, 2, replac...","[rs-28_sarmat, missile_dub, satan_2, replace_s...",rs-28_sarmat missile_dub satan_2 replace_ss-18...


In [ ]:
df.to_csv('processed_fake_news.csv') # saving the preprocessed data so that the model can be build.